In [199]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from sklearn.inspection import permutation_importance

from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

# 1. 문제 정의

# 2. 데이터 수집

## (1) 데이콘 기본 데이터

In [200]:
train = pd.read_csv('data/train.csv').drop(['index'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# 3. 탐색적 데이터 분석

# 4. 변수 조정

In [201]:
# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month/income_total'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['DAYS_BIRTH_week/income_total'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month/income_total'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED_week/income_total'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED
merge_data['before_EMPLOYED/income_total'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['before_EMPLOYED_month/income_total'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['before_EMPLOYED_week/income_total'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# 총 수익을 가족 수로 나누기
merge_data['income_total/family_size'] = merge_data['income_total'] / merge_data['family_size']

merge_data['child_num/income_total'] = merge_data['child_num'] / merge_data['income_total']
merge_data['family_size/income_total'] = merge_data['family_size'] / merge_data['income_total']
merge_data['DAYS_BIRTH/income_total'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/income_total'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
#merge_data['begin_month/income_total'] = merge_data['begin_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/DAYS_BIRTH'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# 소득 skewed-data 처리
#merge_data['log1p_income_total'] = np.log1p(merge_data['income_total'])
# merge_data['log_income_total'] = np.log(merge_data['income_total'])
# merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
# merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

train = train[train['child_num']<=6].reset_index(drop=True) # 아이의 수가 7명 이상인 데이터 제거

## 인코딩

remove_features = [2, 4, 23]
train = train.drop(train.columns[remove_features], axis=1)
test = test.drop(test.columns[remove_features], axis=1)

In [202]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [203]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [204]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']


## Permutation Importance

변수 하나씩 제거하면서 성능 체크<br>
제거하여 성능이 나쁘게 나온 것들은 리스트에 따로 저장해두기

In [164]:
train_x = train_oh.drop(['credit'], axis=1)
train_y = train_oh['credit']
test_x = test_oh.copy()

In [117]:
perm_importance_dict = {}
lucky_seeds=[42, 2019,91373] # Lucky seed 늘려가면서 하기
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True) # CV 늘려가면서 하기
    cv=np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        lgbmodel = LGBMClassifier(objective='multiclass', n_estimators=10000, random_state=seed)
        perm = PermutationImportance(clf, scoring='neg_log_loss', random_state=seed).fit(x_val, y_val)
        
        perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                          index=val_X.columns.tolist()).sort_index()
        perm_importance_dict['seed'+str(seed)+' fold'+str(n)] = perm_importance_df

In [132]:
perm_importance_lgb = list(perm_importance_dict.values())[0]
del perm_importance_dict[list(perm_importance_dict.keys())[0]]
for i, data in perm_importance_dict.items():
    perm_importance_lgb = pd.concat([perm_importance_lgb, data], axis=1)
perm_importance_lgb

,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance,importance
DAYS_BIRTH,0.044746,0.042182,0.041731,0.043062,4.101402e-02,0.042339,0.041120,0.042147,0.043916,0.043490,0.041655,0.043423,0.043797,0.043601,4.201397e-02
DAYS_BIRTH/income_total,0.022819,0.023279,0.022296,0.024382,2.315146e-02,0.023752,0.022141,0.023708,0.022226,0.023011,0.023122,0.021882,0.024767,0.023275,2.380013e-02
DAYS_BIRTH_month,0.007404,0.007679,0.008794,0.008370,7.839389e-03,0.008227,0.008073,0.007050,0.008299,0.008436,0.008167,0.007723,0.008038,0.008420,7.783715e-03
DAYS_BIRTH_month/income_total,0.024522,0.024086,0.023204,0.024768,2.386504e-02,0.023524,0.023610,0.023077,0.023866,0.024114,0.024019,0.023510,0.024667,0.024546,2.517036e-02
DAYS_BIRTH_week,0.002778,0.003363,0.002934,0.002801,3.194540e-03,0.003262,0.002684,0.003340,0.003105,0.003416,0.003328,0.003008,0.003540,0.002861,3.009580e-03
DAYS_BIRTH_week/income_total,0.021009,0.020702,0.020045,0.020525,2.062903e-02,0.020587,0.021097,0.020257,0.020509,0.019911,0.020482,0.021235,0.021266,0.020068,1.978554e-02
DAYS_EMPLOYED,0.023522,0.023764,0.021521,0.023723,2.069835e-02,0.023254,0.023049,0.021280,0.022362,0.021619,0.020495,0.023577,0.023569,0.020404,2.378029e-02
DAYS_EMPLOYED/DAYS_BIRTH,0.037732,0.035746,0.033246,0.035982,3.269973e-02,0.036190,0.036124,0.034449,0.034965,0.032289,0.035456,0.035956,0.035600,0.033267,3.512550e-02
DAYS_EMPLOYED/income_total,0.016806,0.016394,0.014888,0.014395,1.521781e-02,0.016382,0.015062,0.016184,0.015563,0.015744,0.015746,0.016533,0.016611,0.013939,1.653330e-02
DAYS_EMPLOYED_month,0.009072,0.010568,0.008361,0.009449,8.189283e-03,0.010054,0.009451,0.009410,0.009100,0.009221,0.009318,0.008645,0.009605,0.009471,8.985124e-03


In [191]:
remove_features = list(perm_importance_lgb.mean(axis=1).sort_values(ascending=False).index)[-4:]

In [87]:
import eli5
from eli5.sklearn import PermutationImportance 

train_X, val_X, train_y, val_y = train_test_split(train_x, train_y, stratify=train_y, random_state=10086)
clf=LGBMClassifier()
clf.fit(train_X, train_y)
y_pred=clf.predict_proba(val_X)

perm = PermutationImportance(clf, scoring='neg_log_loss', random_state=10086).fit(val_X, val_y)
eli5.show_weights(perm, top = 80, feature_names = val_X.columns.tolist())

Weight,Feature
0.2638 ± 0.0123,begin_month
0.0232 ± 0.0024,DAYS_BIRTH
0.0203 ± 0.0012,DAYS_EMPLOYED/DAYS_BIRTH
0.0128 ± 0.0011,before_EMPLOYED_month/income_total
0.0113 ± 0.0018,DAYS_BIRTH_month/income_total
0.0111 ± 0.0010,DAYS_BIRTH/income_total
0.0107 ± 0.0022,before_EMPLOYED/income_total
0.0104 ± 0.0011,DAYS_EMPLOYED
0.0103 ± 0.0010,DAYS_BIRTH_week/income_total
0.0101 ± 0.0013,DAYS_EMPLOYED_month/income_total


In [8]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# cv = np.zeros((train_x.shape[0], 3))
# for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
#     x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#     y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
#     lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
#     lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
#     cv[val_idx] = lgbm.predict_proba(x_val)
# Initial_log_loss = log_loss(train_y, cv)
# print(f'Initial_multi_logloss: {Initial_log_loss}')

# remove_features = []
# for i in range(1, 2):
#     for j in combinations(list(range(0, train_x.shape[1])), i):
#         train_new_x = train_x.drop(train_x.columns[list(j)], axis=1)
        
#         kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
#         cv = np.zeros((train_new_x.shape[0], 3))
#         for n, (train_idx, val_idx) in enumerate(kfold.split(train_new_x, train_y)):
#             x_train, x_val = train_new_x.iloc[train_idx], train_new_x.iloc[val_idx]
#             y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
#             lgbm = LGBMClassifier(n_estimators=1000, objective='multiclass')
#             lgbm.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None)
#             cv[val_idx] = lgbm.predict_proba(x_val)
#         print(f'{j} multi_logloss: {log_loss(train_y, cv)}')
#         if log_loss(train_y, cv) < Initial_log_loss:
#             remove_features.append(list(j)[0])

## 저장한 변수 지우는 코드

In [9]:
# remove_features = [1, 3, 4, 8, 11, 32]
# train = train.drop(train.columns[remove_features], axis=1)
# test = test.drop(test.columns[remove_features], axis=1)

## 다시 인코딩

In [182]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [183]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [184]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']


# 6. 모델 학습

In [185]:
pred_dict = {}
pred_test_dict = {}

## (1) Lightgbm

In [206]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

### Parameter Tuning

In [15]:
# SEED=0

# # Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
# def score(params):
#     print("Training with params: ")
#     print(params)
    
#     kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
#     cv = np.zeros((train_x.shape[0], 3))

#     for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
#         x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#         y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

#         lgbmodel = LGBMClassifier(**params)

#         lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 
#         cv[val_idx, :] = lgbmodel.predict_proba(x_val)
#         #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
#     print('multi_logloss:', log_loss(train_y, cv))
#     score = log_loss(train_y, cv)
#     return {'loss': score, 'status': STATUS_OK}

# # Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
# def optimize(random_state=SEED):
    
# #     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
# #          'eta': 0.01, 'min_child_weight': 3,
# #          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
# #         }
#     space = {
#         #'learning_rate': hp.quniform('learning_rate', 0.004, 0.005, 0.001),
#         'learning_rate' : 0.004,
#         #'num_leaves': scope.int(hp.quniform('num_leaves', 1000, 1200, 50)),
#         'num_leaves' : 1000,
#         #'min_child_weight': hp.quniform('min_child_weight', 1, 3, 1),
#         'min_child_weight' : 2,
#         #'subsample': hp.quniform('subsample', 0.8, 1, 0.05),
#         'subsample' : 1,
#         #'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.61, 0.01),
#         'colsample_bytree' : 0.6,
#         #'reg_alpha': hp.quniform('lambda_l1', 0.9, 1, 0.01),
#         'reg_alpha' : 0.94,
#         #'reg_lambda': hp.quniform('lambda_l2', 0.8, 1, 0.02),
#         'reg_lambda' : 0.98,
#         'max_depth' : -1,
#         'n_estimators' : 5000,
#         'objective' : 'multiclass',
#         'num_class' : 3,
#         'seed': SEED,
#     }
#     # Use the fmin function from Hyperopt to find the best hyperparameters
#     best = fmin(score, space, algo=tpe.suggest, 
#                 # trials=trials, 
#                 max_evals=12)
#     return best

# best_hyperparams = optimize()
# print("The best hyperparameters are: ", "\n")
# print(best_hyperparams)

### 3 seeds x 10 folds

In [16]:
lucky_seeds=[42,2019,91373] # Lucky seed 늘려가면서 하기
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # CV 늘려가면서 하기
    cv=np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        lgbmodel = LGBMClassifier(learning_rate=0.004, objective='multiclass', n_estimators=10000, num_leaves=1000, 
                                  max_depth=-1, min_child_weight=2, colsample_bytree=0.6, reg_alpha=0.94, reg_lambda=0.98,
                                   n_jobs=-1, random_state=seed)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 
        #joblib.dump(lgbmodel, f'./pred_pkl/LGB_{n+1}_fold_{seed}_seed_lgb.pkl')

        # CROSS-VALIDATION , EVALUATE CV
        cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_x) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
    pred_dict['lgb'+str(seed)] = cv
    pred_test_dict['lgb'+str(seed)] = pred_test
        
    print('multi_logloss :', log_loss(train_y, cv))

multi_logloss : 0.6873967339120363
multi_logloss : 0.6866276155463265
multi_logloss : 0.6847636925541635


lgbmodels_path = os.listdir('./pred_pkl/')
lgbmodels_list = [x for x in lgbmodels_path if x.endswith("lgb.pkl")]
assert len(lgbmodels_list) == 15
lgb_preds = np.zeros((test_x.shape[0], 3))

for m in lgbmodels_list:
    lgbmodel = joblib.load('./pred_pkl/'+m)
    lgb_preds_proba = lgbmodel.predict_proba(test)
    lgb_preds += lgb_preds_proba/15

## (2) XGBoost

원핫인코딩된 feature로 만들어주기 **꼭 밑에 코드 실행하고 XGBoost랑 Randomforest 돌리기!!**

In [17]:
train_x = train_oh.drop(['credit'], axis=1)
train_y = train_oh['credit']
test_x = test_oh.copy()

### Parameter Tuning (hyperopt)

In [18]:
# SEED=0

# # Hyperopt의 metric함수를 StratifiedKFold(cv=5)로 구하기
# def score(params):
#     print("Training with params: ")
#     print(params)
    
#     kfold = StratifiedKFold(n_splits=5, random_state = SEED, shuffle = True)
#     cv = np.zeros((train_x.shape[0], 3))

#     for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
#         x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
#         y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
#         dtrain = xgb.DMatrix(x_train, label=y_train)
#         dvalid = xgb.DMatrix(x_val, label=y_val)
#         watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

#         xgbmodel = xgb.train(params, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)
#         cv[val_idx, :] = xgbmodel.predict(dvalid)
#         print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
#     print('multi_logloss:', log_loss(train_y, cv))
#     score = log_loss(train_y, cv)
#     return {'loss': score, 'status': STATUS_OK}

# # Hyperopt의 범위를 지정해주고 max_evals만큼 반복한 후 최적의 파라미터를 반환
# def optimize(random_state=SEED):
    
# #     param = {'objective':'multi:softprob', 'seed':SEED, 'num_class': 3, 'eval_metric':'mlogloss', 
# #          'eta': 0.01, 'min_child_weight': 3,
# #          'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
# #         }
#     space = {
#         #'eta': hp.quniform('eta', 0.006, 0.007, 0.001),
#         'eta' : 0.005,
#         #'max_depth':  hp.choice('max_depth', np.arange(20, 30, dtype=int)),
#         #'min_child_weight': hp.quniform('min_child_weight', 4, 5, 1),
#         'min_child_weight': 4,
#         #'subsample': hp.quniform('subsample', 0.72, 0.8, 0.02),
#         'subsample': 0.72,
#         #'gamma': hp.quniform('gamma', 0.6, 0.8, 0.05),
#         'gamma': 0.7,
#         #'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
#         'colsample_bytree': 0.45,
#         #'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 0.15, 0.01),
#         'colsample_bylevel': 0.15,
#         'max_depth' : 100,
#         'eval_metric': 'mlogloss',
#         'objective' : 'multi:softprob',
#         'num_class' : 3,
#         'seed': SEED,
#     }
#     # Use the fmin function from Hyperopt to find the best hyperparameters
#     best = fmin(score, space, algo=tpe.suggest, 
#                 # trials=trials, 
#                 max_evals=10)
#     return best

# best_hyperparams = optimize()
# print("The best hyperparameters are: ", "\n")
# print(best_hyperparams)

### 3 seeds x 10 folds

In [27]:
lucky_seeds=[42, 2019, 91373] # 늘려가면서
xgtest = xgb.DMatrix(test_x)
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # 늘려가면서
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

        param = {'colsample_bylevel': 0.15, 'colsample_bytree': 0.45, 'eta': 0.005, 'eval_metric': 'mlogloss', 
         'gamma': 0.7, 'max_depth': 100, 'min_child_weight': 4, 'num_class': 3, 
         'objective': 'multi:softprob', 'seed': 0, 'subsample': 0.72}
        xgbmodel = xgb.train(param, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=None)
        #joblib.dump(xgbmodel, f'./pred_pkl/XGB_{n+1}_fold_{seed}_seed_xgb.pkl')

        cv[val_idx, :] = xgbmodel.predict(dvalid)
        #print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += xgbmodel.predict(xgtest) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
        
    pred_dict['xgb'+str(seed)] = cv
    pred_test_dict['xgb'+str(seed)] = pred_test
    print('multi_logloss:', log_loss(train_y, cv))

multi_logloss: 0.6794795536262115
multi_logloss: 0.6785301842295772
multi_logloss: 0.6768298757692274


xgbmodels_path = os.listdir('./pred_pkl/')
xgbmodels_list = [x for x in xgbmodels_path if x.endswith("xgb.pkl")]
assert len(xgbmodels_list) == 15
xgb_preds = np.zeros((test_x.shape[0], 3))
xgtest = xgb.DMatrix(test_X)

for m in xgbmodels_list:
    xgbmodel = joblib.load('./pred_pkl/'+m)
    xgb_preds_proba = xgbmodel.predict_proba(xgtest
       xgb_preds += xgb_preds_proba/15

## (3) Random Forest

### GridSearchCV

params = {'max_depth': [55, 60, 65] # 튜닝할 파라미터 삽입
            }

rf_clf = RandomForestClassifier(random_state = 0, n_estimators = 1000, 
                                min_samples_leaf=2, min_samples_split=2,
                                criterion='entropy', n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
grid_cv.fit(df_train, y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

### 3 seeds, 10 folds

In [ ]:
lucky_seeds=[42,2019,91373] # 늘려가면서
for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True) # 늘려가면서
    cv = np.zeros((train_x.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        rfmodel = RandomForestClassifier(n_estimators=1200, criterion='entropy', max_depth=60,
                                         min_samples_leaf=2, min_samples_split=2,
                                         random_state=seed)
        rfmodel.fit(x_train, y_train)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = rfmodel.predict_proba(x_val)      
        print(f'fold{n+1} multi_logloss: {log_loss(y_val, cv[val_idx, :])}')
        pred_test += rfmodel.predict_proba(test_x) / 10 # CV 바꾸면 이 숫자도 똑같이 바꿔야함
        
    pred_dict['rf'+str(seed)] = cv
    pred_test_dict['rf'+str(seed)] = pred_test
    print('multi_logloss :', log_loss(train_y, cv))

rfmodels_path = os.listdir('./pred_pkl/')
rfmodels_list = [x for x in rfmodels_path if x.endswith("rf.pkl")]
assert len(rfmodels_list) == 15
rf_preds = np.zeros((test_x.shape[0], 3))

for m in rfmodels_list:
    rfmodel = joblib.load('./pred_pkl/'+m)
    rf_preds_proba = rfmodel.predict_proba(test_x)
    rf_preds += rf_preds_proba/15

## (4) Catboost (성능X)

lucky_seeds=[42,2019,91373]
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        _train = Pool(x_train, label=y_train)
        _valid = Pool(x_val, label=y_val)

        catmodel =  CatBoostClassifier(loss_function='MultiClass', early_stopping_rounds=50, 
                                       random_state=seed, learning_rate=0.02, iterations=100000
                                       #task_type="GPU"
                                      )
        
        catmodel.fit(_train, eval_set=_valid, use_best_model=True, verbose=2000)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = catmodel.predict_proba(x_val)        
        pred_test += catmodel.predict_proba(test_x) / 5
        
    pred_dict['cat'+str(i+1)] = cv
    pred_test_dict['cat'+str(i+1)] = pred_test
    print('multi_logloss :', log_loss(true, cv))

## (4) Stacking (AutoLGB)

### 27features = 3seed(42, 2019, 91373) x 3model(lgb, xgb, rf) x 3class(0, 1, 2)

In [ ]:
X_train = pd.DataFrame(np.hstack([x for _, x in pred_dict.items()]))
X_test = pd.DataFrame(np.hstack([x for _, x in pred_test_dict.items()]))

pred = np.zeros((X_train.shape[0], 3), dtype=float)
pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
#kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42) # 이건 CV 너무 크게하면 안됨, 3~6까지 테스트해보면 좋을듯

for i_cv, (i_trn, i_val) in enumerate(cv.split(X_train, train_y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': 3}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X_train.iloc[i_trn], train_y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        print(params)
        clf.fit(X_train.iloc[i_trn], train_y[i_trn])
    else:
        train_data = lgb.Dataset(X_train[features].iloc[i_trn], label=train_y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    pred[i_val] = clf.predict(X_train[features].iloc[i_val])
    pred_test += clf.predict(X_test[features]) / 4

In [ ]:
print(f'CV Log Loss: {log_loss(train_y, pred):.6f}')

# 결과 제출

In [ ]:
submission = sample_submission.copy()
submission.iloc[:, 1:] = pred_test

In [ ]:
submission.to_csv('submission/submission.csv', index=False)